In [7]:
# pip install python-dotenv
# pip install spotipy --upgrade
# pip install python-vlc
import os
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import spotipy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# import vlc
import json
import time

load_dotenv()

# client_credentials_manager = SpotifyClientCredentials(client_id=os.environ.get("SPOTIPY_CLIENT_ID"),
#                                                       client_secret=os.environ.get("SPOTIPY_CLIENT_SECRET"))

# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# https://open.spotify.com/playlist/37i9dQZF1E38k2WiBlq4DF?si=96e4439556ff412c
# https://open.spotify.com/track/0rc3gOGiTLUz9rZ3ypsYW7?si=721667e208cb4c64
# results = sp.playlist("37i9dQZF1E38k2WiBlq4DF")
# results = sp.track("66lF1QWtLZXJofJ3x388RV")

False

In [8]:
def get_genres_by_track_id(track_id):

    track_data = sp.track(track_id)

    artist_ids = []

    for artist in track_data["artists"]:
        artist_ids.append(artist["id"])

    artists_data = sp.artists(artist_ids)

    genres = []

    for artist in artists_data["artists"]:
        genres += artist["genres"]
        break

    return genres

def play_music(final_df):
    #ลองเล่นเพลงที่ได้จาก preview_url
    song = str(final_df['preview_url'].sample().values[0])
    p = vlc.MediaPlayer(song)

    #ตั้งความดังเสียง
    p.audio_set_volume(50)

    p.play()

def get_genres():
    # Open genres file
    try:
        with open('genres.json', 'r') as infile:
            valid_genres = json.load(infile)
    except FileNotFoundError:
        print("Couldn't find genres file!")
        sys.exit(1)
    
    return valid_genres

def create_csv(song_meta,genre,limit):
    index = 0
    song_meta_df = pd.DataFrame.from_dict(song_meta)
 
    # check the song feature
    features = sp.audio_features(song_meta['id'][0:50])
    for i in range(1,limit//50):
        features += sp.audio_features(song_meta['id'][50*i:50*i + 50])
        
    # change dictionary to dataframe
    features_df = pd.DataFrame.from_dict(features)

    # combine two dataframe
    final_df = song_meta_df.merge(features_df)

    final_df = final_df.drop(columns=['type','uri', 'track_href', 'time_signature'])

    final_df.to_csv("csv-file/" + genre + ".csv",index=False)

def check_file(genre):
    path  = "csv-file/" + genre + ".csv"
    check_file = os.path.isfile(path)
    return check_file

In [9]:
genres_list = get_genres()
ids=[]
limit_per_genres = 250
genres_not_create = []

In [10]:
for genre in genres_list:
    if check_file(genre):
        ids += list(pd.read_csv("csv-file/" + genre + ".csv").id)
        continue
    break

In [11]:
len(genres_list)

126

In [12]:
len(ids)

2250

In [13]:
for genre in genres_list:
    offset = 0
    song_meta={'id':[],'preview_url':[],'genre':[]}
    
    if check_file(genre):
        print("=======> have file {}.csv".format(genre))
        continue
        
    while list(song_meta.items())[2][1].count(genre) < limit_per_genres:
        
        print("{} : {}".format(genre,list(song_meta.items())[2][1].count(genre)))
        time.sleep(1)
        index = len(ids)
        try:
            results = sp.search(q="genre:"+genre,limit=50,offset=offset,type="track")
        except:
            break
            
        if (len(results['tracks']['items']) == 0):
            print("=======> result for search genre {} is 0 track".format(genre))
            break
        
        offset += 50
        # create a list of song ids
        for item in results['tracks']['items']:
            if item['id'] in ids:
                continue
            track = item['id']
            ids.append(track)
            
            if list(song_meta.items())[2][1].count(genre) == limit_per_genres:
                print("{} : {}".format(genre,list(song_meta.items())[2][1].count(genre)))
                break
            
            if item['preview_url'] == None:
                continue
                
            # song id
            song_meta['id'].append(item['id'])

            # song preview
            preview_url = item['preview_url']
            song_meta['preview_url'] += [preview_url]

            # song genres
            song_genre = genre
            song_meta["genre"] += [song_genre]
        
#         for song_id in ids[index:]:
            
#             if list(song_meta.items())[2][1].count(genre) == limit_per_genres:
#                 print("{} : {}".format(genre,list(song_meta.items())[2][1].count(genre)))
#                 break
#             # get song's meta data
#             time.sleep(0.2)
#             meta = sp.track(song_id)
            
#             if meta['preview_url'] == None:
#                 continue
                
#             # song id
#             song_meta['id'].append(song_id)

#             # song preview
#             preview_url = meta['preview_url']
#             song_meta['preview_url'] += [preview_url]

#             # song genres
#             song_genre = genre
#             song_meta["genre"] += [song_genre]
        
    if list(song_meta.items())[2][1].count(genre) == 0:
        print("=======> can't create file {}.csv".format(genre))
        genres_not_create.append(genre)
        continue
    
    print("=======> create csv {} {}".format(genre,list(song_meta.items())[2][1].count(genre)))
    create_csv(song_meta,genre,limit_per_genres)

=======> have file acoustic.csv
=======> have file afrobeat.csv
=======> have file alt-rock.csv
=======> have file alternative.csv
=======> have file ambient.csv
=======> have file anime.csv
=======> have file black-metal.csv
=======> have file bluegrass.csv
=======> have file blues.csv
bossanova : 0
=======> can't create file bossanova.csv
=======> have file brazil.csv
=======> have file breakbeat.csv
=======> have file british.csv
=======> have file cantopop.csv
=======> have file chicago-house.csv
=======> have file children.csv
=======> have file chill.csv
=======> have file classical.csv
=======> have file club.csv
=======> have file comedy.csv
=======> have file country.csv
=======> have file dance.csv
=======> have file dancehall.csv
=======> have file death-metal.csv
=======> have file deep-house.csv
=======> have file detroit-techno.csv
=======> have file disco.csv
disney : 0
=======> can't create file disney.csv
=======> have file drum-and-bass.csv
=======> have file dub.csv


In [14]:
len(genres_not_create)

13

In [15]:
genres_not_create

['bossanova',
 'disney',
 'holidays',
 'metal-misc',
 'movies',
 'new-release',
 'philippines-opm',
 'post-dubstep',
 'rainy-day',
 'road-trip',
 'soundtracks',
 'summer',
 'work-out']

In [16]:
file_path = "csv-file"
#list all the files from the directory
file_list = os.listdir(file_path)

df_append = pd.DataFrame()
#append all files together
for file in file_list:
            df_temp = pd.read_csv("csv-file/" + file)
            df_append = df_append.concat([df_append,df_temp], ignore_index=True)
df_append.head()

AttributeError: 'DataFrame' object has no attribute 'concat'

In [58]:
len(df_append), len(file_list) * 250

(28250, 28250)

In [59]:
df_append.to_csv("csv-file/" + "all_genre" + ".csv",index=False)